In [1]:
!wget https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar -O tabular-dl-tabr.tar.gz
!tar -xvf tabular-dl-tabr.tar.gz

--2024-11-16 12:01:44--  https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar
Resolving huggingface.co (huggingface.co)... 3.171.171.128, 3.171.171.6, 3.171.171.104, ...
Connecting to huggingface.co (huggingface.co)|3.171.171.128|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/1b/25/1b25d6e2556c827abfaf6ba9da6021338691cafc39fe9d77986955ae0a69243d/0d74e4b96febb48fbe4dd2d851f8638b5738354f82c3183f92adbd2abf2f256b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data.tar%3B+filename%3D%22data.tar%22%3B&response-content-type=application%2Fx-tar&Expires=1732017704&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjAxNzcwNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xYi8yNS8xYjI1ZDZlMjU1NmM4MjdhYmZhZjZiYTlkYTYwMjEzMzg2OTFjYWZjMzlmZTlkNzc5ODY5NTVhZTBhNjkyNDNkLzBkNzRlNGI5NmZlYmI0OGZiZTRkZDJkODUxZjg2MzhiNTczODM1NGY4MmMzMTgzZjkyYWRiZDJhYmYyZjI1Nm

In [2]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.6 MB/s eta 0:00:00


In [3]:
!pip install psutil

In [4]:
import pandas as pd
import numpy as np
#import optuna
import torch.optim
import psutil

from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import QuantileTransformer

In [ ]:
clf = TabNetClassifier()

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [5]:
X_train = np.load('/content/data/classif-num-large-0-jannis/X_num_train.npy')
y_train = np.load('/content/data/classif-num-large-0-jannis/Y_train.npy')

X_valid = np.load('/content/data/classif-num-large-0-jannis/X_num_val.npy')
y_valid = np.load('/content/data/classif-num-large-0-jannis/Y_val.npy')

X_test = np.load('/content/data/classif-num-large-0-jannis/X_num_test.npy')
y_test = np.load('/content/data/classif-num-large-0-jannis/Y_test.npy')

In [6]:
normalizer = QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X_train.shape[0] // 30, 1000), 10),
            subsample=1_000_000_000,
            )
normalizer.fit_transform(X_train)

array([[-0.3712373 , -0.70553   ,  0.12831734, ..., -0.08481442,
        -2.9971676 , -1.6137935 ],
       [-0.15916775, -0.04727262, -0.65804297, ..., -1.5936546 ,
         0.5697825 ,  1.0989051 ],
       [-0.02582722, -0.28874987,  0.04769165, ..., -0.91025573,
         0.85823333, -0.30053744],
       ...,
       [ 0.18292761, -0.06654125,  0.5293011 , ...,  0.5947219 ,
        -0.39786035, -1.2680134 ],
       [-0.48698598,  0.01631011, -1.0887225 , ...,  1.1580657 ,
         2.256787  ,  0.10056727],
       [ 1.9972794 ,  0.16509548,  5.1993375 , ..., -1.8457546 ,
        -0.15130426, -0.12106627]], dtype=float32)

In [ ]:
#Baseline
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
)

epoch 0  | loss: 0.68042 | train_accuracy: 0.56711 | valid_accuracy: 0.5797  |  0:00:04s
epoch 1  | loss: 0.59531 | train_accuracy: 0.68479 | valid_accuracy: 0.68777 |  0:00:06s
epoch 2  | loss: 0.56477 | train_accuracy: 0.70473 | valid_accuracy: 0.70648 |  0:00:08s
epoch 3  | loss: 0.54832 | train_accuracy: 0.71741 | valid_accuracy: 0.72134 |  0:00:11s
epoch 4  | loss: 0.53904 | train_accuracy: 0.72468 | valid_accuracy: 0.72597 |  0:00:13s
epoch 5  | loss: 0.53768 | train_accuracy: 0.72659 | valid_accuracy: 0.73389 |  0:00:16s
epoch 6  | loss: 0.53323 | train_accuracy: 0.73299 | valid_accuracy: 0.73157 |  0:00:18s
epoch 7  | loss: 0.52909 | train_accuracy: 0.73661 | valid_accuracy: 0.73311 |  0:00:21s
epoch 8  | loss: 0.52517 | train_accuracy: 0.73877 | valid_accuracy: 0.73736 |  0:00:23s
epoch 9  | loss: 0.5211  | train_accuracy: 0.74163 | valid_accuracy: 0.74527 |  0:00:26s
epoch 10 | loss: 0.51948 | train_accuracy: 0.74366 | valid_accuracy: 0.73755 |  0:00:28s
epoch 11 | loss: 0.51

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
def objective(trial):
    params = {
        'n_d': trial.suggest_int('n_d', 8, 64),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        'momentum': trial.suggest_float('momentum', 0.01, 0.4),
    }

    model = TabNetClassifier(**params, n_a=params['n_d'])
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=['accuracy'], patience=10)

    # Evaluate model performance
    score = model.best_cost  # or any other metric

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params

[I 2024-07-06 09:56:15,657] A new study created in memory with name: no-name-ea3d82f2-9a4d-4d33-a7ad-f8c74223c5b6
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.83397 | val_0_accuracy: 0.52721 |  0:00:04s
epoch 1  | loss: 0.64971 | val_0_accuracy: 0.64435 |  0:00:08s
epoch 2  | loss: 0.63275 | val_0_accuracy: 0.54786 |  0:00:12s
epoch 3  | loss: 0.63269 | val_0_accuracy: 0.65708 |  0:00:16s
epoch 4  | loss: 0.5884  | val_0_accuracy: 0.68313 |  0:00:21s
epoch 5  | loss: 0.57632 | val_0_accuracy: 0.67927 |  0:00:25s
epoch 6  | loss: 0.56808 | val_0_accuracy: 0.70668 |  0:00:29s
epoch 7  | loss: 0.57366 | val_0_accuracy: 0.70301 |  0:00:33s
epoch 8  | loss: 0.56919 | val_0_accuracy: 0.71054 |  0:00:37s
epoch 9  | loss: 0.56322 | val_0_accuracy: 0.70571 |  0:00:41s
epoch 10 | loss: 0.55814 | val_0_accuracy: 0.71478 |  0:00:46s
epoch 11 | loss: 0.55542 | val_0_accuracy: 0.71768 |  0:00:50s
epoch 12 | loss: 0.55922 | val_0_accuracy: 0.70783 |  0:00:54s
epoch 13 | loss: 0.55711 | val_0_accuracy: 0.72231 |  0:00:58s
epoch 14 | loss: 0.54658 | val_0_accuracy: 0.71729 |  0:01:02s
epoch 15 | loss: 0.54795 | val_0_accuracy: 0.71999 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 10:03:07,828] Trial 0 finished with value: 0.7879197221150135 and parameters: {'n_d': 24, 'n_steps': 7, 'gamma': 1.7164609558573756, 'n_independent': 5, 'n_shared': 1, 'momentum': 0.29796124475339764}. Best is trial 0 with value: 0.7879197221150135.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.06396 | val_0_accuracy: 0.55751 |  0:00:07s
epoch 1  | loss: 0.66865 | val_0_accuracy: 0.63161 |  0:00:15s
epoch 2  | loss: 0.67293 | val_0_accuracy: 0.51679 |  0:00:23s
epoch 3  | loss: 0.63929 | val_0_accuracy: 0.63933 |  0:00:31s
epoch 4  | loss: 0.61342 | val_0_accuracy: 0.6868  |  0:00:39s
epoch 5  | loss: 0.59793 | val_0_accuracy: 0.68777 |  0:00:47s
epoch 6  | loss: 0.60585 | val_0_accuracy: 0.66847 |  0:00:55s
epoch 7  | loss: 0.60801 | val_0_accuracy: 0.68178 |  0:01:03s
epoch 8  | loss: 0.59704 | val_0_accuracy: 0.67831 |  0:01:11s
epoch 9  | loss: 0.59554 | val_0_accuracy: 0.69452 |  0:01:18s
epoch 10 | loss: 0.59012 | val_0_accuracy: 0.69336 |  0:01:26s
epoch 11 | loss: 0.5831  | val_0_accuracy: 0.71343 |  0:01:34s
epoch 12 | loss: 0.57032 | val_0_accuracy: 0.70591 |  0:01:42s
epoch 13 | loss: 0.57041 | val_0_accuracy: 0.7032  |  0:01:50s
epoch 14 | loss: 0.56545 | val_0_accuracy: 0.71671 |  0:01:58s
epoch 15 | loss: 0.56601 | val_0_accuracy: 0.71131 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 10:09:23,562] Trial 1 finished with value: 0.7333076032419915 and parameters: {'n_d': 15, 'n_steps': 10, 'gamma': 1.618774420256689, 'n_independent': 4, 'n_shared': 5, 'momentum': 0.344054743175641}. Best is trial 0 with value: 0.7879197221150135.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 15:25:26,622] Trial 97 finished with value: 0.7825164029332304 and parameters: {'n_d': 18, 'n_steps': 3, 'gamma': 1.9161103342861856, 'n_independent': 5, 'n_shared': 4, 'momentum': 0.24730772817812335}. Best is trial 27 with value: 0.7994982632188344.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.64612 | val_0_accuracy: 0.64821 |  0:00:01s
epoch 1  | loss: 0.57632 | val_0_accuracy: 0.67599 |  0:00:03s
epoch 2  | loss: 0.56465 | val_0_accuracy: 0.69626 |  0:00:05s
epoch 3  | loss: 0.55132 | val_0_accuracy: 0.71169 |  0:00:06s
epoch 4  | loss: 0.5433  | val_0_accuracy: 0.72887 |  0:00:08s
epoch 5  | loss: 0.53007 | val_0_accuracy: 0.7335  |  0:00:10s
epoch 6  | loss: 0.52348 | val_0_accuracy: 0.73562 |  0:00:12s
epoch 7  | loss: 0.51379 | val_0_accuracy: 0.74276 |  0:00:13s
epoch 8  | loss: 0.51203 | val_0_accuracy: 0.74527 |  0:00:15s
epoch 9  | loss: 0.50486 | val_0_accuracy: 0.75569 |  0:00:17s
epoch 10 | loss: 0.50356 | val_0_accuracy: 0.75801 |  0:00:19s
epoch 11 | loss: 0.49786 | val_0_accuracy: 0.75145 |  0:00:20s
epoch 12 | loss: 0.49537 | val_0_accuracy: 0.75994 |  0:00:22s
epoch 13 | loss: 0.49267 | val_0_accuracy: 0.7609  |  0:00:24s
epoch 14 | loss: 0.49207 | val_0_accuracy: 0.76013 |  0:00:25s
epoch 15 | loss: 0.49297 | val_0_accuracy: 0.76746 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 15:26:50,911] Trial 98 finished with value: 0.7925511385565419 and parameters: {'n_d': 22, 'n_steps': 3, 'gamma': 1.798917586063842, 'n_independent': 2, 'n_shared': 2, 'momentum': 0.05501465960487304}. Best is trial 27 with value: 0.7994982632188344.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.00447 | val_0_accuracy: 0.53724 |  0:00:05s
epoch 1  | loss: 0.69652 | val_0_accuracy: 0.59629 |  0:00:10s
epoch 2  | loss: 0.63508 | val_0_accuracy: 0.6567  |  0:00:16s
epoch 3  | loss: 0.61827 | val_0_accuracy: 0.62138 |  0:00:21s
epoch 4  | loss: 0.61876 | val_0_accuracy: 0.6592  |  0:00:27s
epoch 5  | loss: 0.60572 | val_0_accuracy: 0.66731 |  0:00:32s
epoch 6  | loss: 0.60811 | val_0_accuracy: 0.67734 |  0:00:38s
epoch 7  | loss: 0.59938 | val_0_accuracy: 0.66885 |  0:00:43s
epoch 8  | loss: 0.60003 | val_0_accuracy: 0.68178 |  0:00:48s
epoch 9  | loss: 0.5916  | val_0_accuracy: 0.67522 |  0:00:54s
epoch 10 | loss: 0.59661 | val_0_accuracy: 0.6731  |  0:00:59s
epoch 11 | loss: 0.58825 | val_0_accuracy: 0.68641 |  0:01:05s
epoch 12 | loss: 0.58892 | val_0_accuracy: 0.68236 |  0:01:10s
epoch 13 | loss: 0.58825 | val_0_accuracy: 0.67368 |  0:01:15s
epoch 14 | loss: 0.58582 | val_0_accuracy: 0.68275 |  0:01:21s
epoch 15 | loss: 0.57996 | val_0_accuracy: 0.68294 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 15:31:58,223] Trial 99 finished with value: 0.7589733693554612 and parameters: {'n_d': 36, 'n_steps': 6, 'gamma': 1.9085527929229418, 'n_independent': 5, 'n_shared': 5, 'momentum': 0.3823228842489212}. Best is trial 27 with value: 0.7994982632188344.


In [ ]:
best_params

{'n_d': 33,
 'n_steps': 9,
 'gamma': 1.9639007888000015,
 'n_independent': 3,
 'n_shared': 3,
 'momentum': 0.039107717833756696}

In [7]:
best_params = {'n_d': 33,
 'n_steps': 9,
 'gamma': 1.9639007888000015,
 'n_independent': 3,
 'n_shared': 3,
 'momentum': 0.039107717833756696}

In [8]:
from pytorch_tabnet.callbacks import Callback

def print_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"System RAM Usage: {mem_info.rss / (1024 ** 2):.2f} MB")
    if torch.cuda.is_available():
      print(f"Allocated GPU memory: {torch.cuda.memory_allocated() / (1024 ** 2):.2f} MB")
      print(f"Cached GPU memory: {torch.cuda.memory_reserved() / (1024 ** 2):.2f} MB")

class MemoryUsageCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print_memory_usage()

In [9]:
# Train final model using best hyperparameters
best_model = TabNetClassifier(
    **best_params,
    n_a=best_params['n_d'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2)
)

best_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=200,
    patience=16,
    callbacks=[MemoryUsageCallback()]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.10779 | train_accuracy: 0.57704 | valid_accuracy: 0.57893 |  0:00:16s
System RAM Usage: 1222.17 MB
Allocated GPU memory: 23.06 MB
Cached GPU memory: 146.00 MB
epoch 1  | loss: 0.92703 | train_accuracy: 0.62517 | valid_accuracy: 0.63064 |  0:00:26s
System RAM Usage: 1223.57 MB
Allocated GPU memory: 23.06 MB
Cached GPU memory: 148.00 MB
epoch 2  | loss: 0.64951 | train_accuracy: 0.62946 | valid_accuracy: 0.61984 |  0:00:35s
System RAM Usage: 1223.57 MB
Allocated GPU memory: 23.06 MB
Cached GPU memory: 148.00 MB
epoch 3  | loss: 0.63961 | train_accuracy: 0.65536 | valid_accuracy: 0.64068 |  0:00:45s
System RAM Usage: 1223.57 MB
Allocated GPU memory: 23.06 MB
Cached GPU memory: 148.00 MB
epoch 4  | loss: 0.61452 | train_accuracy: 0.67198 | valid_accuracy: 0.6704  |  0:00:54s
System RAM Usage: 1223.57 MB
Allocated GPU memory: 23.06 MB
Cached GPU memory: 148.00 MB
epoch 5  | loss: 0.61148 | train_accuracy: 0.65427 | valid_accuracy: 0.65071 |  0:01:04s
System RAM Usage: 122

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

Best Valid Score: 0.7975685063681976
Test AUC: 0.8620650832214649


In [ ]:
preds = best_model.predict(X_test)

print(classification_report(y_true=y_test, y_pred=preds))

              precision    recall  f1-score   support

           0       0.80      0.77      0.78      6051
           1       0.78      0.81      0.79      6041

    accuracy                           0.79     12092
   macro avg       0.79      0.79      0.79     12092
weighted avg       0.79      0.79      0.79     12092



In [10]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

Best Valid Score: 0.7975685063681976
Test AUC: 0.8620650832214649


In [11]:
fpr, tpr, thresholds = roc_curve(y_score=y_proba[:,1], y_true=y_test)

roc_data = pd.DataFrame({
    'fpr': fpr,
    'tpr': tpr
})

roc_data.to_csv('jannis-TabNet.csv', index=False)